# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-14-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-14-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-07-15 04:21:48,33.93911,67.709953,136643,5923,82586.0,48134.0,Afghanistan,351.011619,4.334653
1,NaN,NaN,NaN,Albania,2021-07-15 04:21:48,41.15330,20.168300,132616,2456,130059.0,101.0,Albania,4608.242407,1.851964
2,NaN,NaN,NaN,Algeria,2021-07-15 04:21:48,28.03390,1.659600,148797,3882,103192.0,41723.0,Algeria,339.323742,2.608924
3,NaN,NaN,NaN,Andorra,2021-07-15 04:21:48,42.50630,1.521800,14167,127,13796.0,244.0,Andorra,18335.598266,0.896449
4,NaN,NaN,NaN,Angola,2021-07-15 04:21:48,-11.20270,17.873900,40327,946,34584.0,4797.0,Angola,122.700271,2.345823


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,125937,127464,129021,130113,131586,131586,131586,134653,134653,136643
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132537,132544,132557,132565,132580,132587,132592,132597,132608,132616
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,141966,142447,143032,143652,144483,145296,146064,146942,147883,148797


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,5283,5360,5415,5477,5561,5561,5561,5791,5791,5923
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3765,3775,3786,3798,3811,3824,3836,3851,3867,3882


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,75314,76218,77280,78521,79446,80226,81077,82586,82586,82586
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,130029,130033,130035,130037,130043,130047,130050,130052,130053,130059
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,98748,99101,99510,99931,100383,100911,101430,102010,102594,103192


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7262,7262,7277,7277,7294,7299,7299,7306,7306,7320
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,22043,22043,22154,22154,22220,22267,22267,22337,22337,22464
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2347,2347,2354,2354,2361,2365,2365,2367,2367,2373


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,315,315,316,316,317,318,318,318,318,321
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,60,60,60,60,60,60,60,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-07-15 04:21:48,32.539527,-86.644082,7320,113,NaN,NaN,"Autauga, Alabama, US",13102.078075,1.543716
693,1075.0,Lamar,Alabama,US,2021-07-15 04:21:48,33.779950,-88.096680,1481,38,NaN,NaN,"Lamar, Alabama, US",10727.997102,2.565834
694,1077.0,Lauderdale,Alabama,US,2021-07-15 04:21:48,34.901719,-87.656247,9754,253,NaN,NaN,"Lauderdale, Alabama, US",10518.823669,2.593808


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33947182,608115,0.0
India,30946074,411408,30104720.0
Brazil,19209729,537394,17128429.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33947182,608115,0.0,33339067.0,2021-07-15 04:21:48,37.950547,-91.419547
India,30946074,411408,30104720.0,429946.0,2021-07-15 04:21:48,23.088275,81.806127
Brazil,19209729,537394,17128429.0,1543906.0,2021-07-15 04:21:48,-12.669522,-48.480493
France,5884395,111609,407965.0,5364821.0,2021-07-15 04:21:48,6.430808,-34.730285
Russia,5785542,142877,5195394.0,447271.0,2021-07-15 04:21:48,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3847746,64012,0.0
Texas,3029676,52727,0.0
Florida,2404895,38157,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3847746,64012,0.0,3783734.0,2021-07-15 04:21:48,37.843962,-120.728594
Texas,3029676,52727,0.0,2976949.0,2021-07-15 04:21:48,31.660643,-98.653069
Florida,2404895,38157,0.0,2366738.0,2021-07-15 04:21:48,28.940755,-82.700744
New York,2122502,53754,0.0,2068748.0,2021-07-15 04:21:48,42.544151,-75.474183
Illinois,1398410,25776,0.0,1372634.0,2021-07-15 04:21:48,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1261068,24569,0.0
Arizona,Maricopa,566087,10370,0.0
Illinois,Cook,558563,10536,0.0
Florida,Miami-Dade,515765,6472,0.0
Texas,Harris,406425,6638,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1261068,24569,0.0,1236499.0,2021-07-15 04:21:48,34.308284,-118.228241,6037.0
Arizona,Maricopa,566087,10370,0.0,555717.0,2021-07-15 04:21:48,33.348359,-112.491815,4013.0
Illinois,Cook,558563,10536,0.0,548027.0,2021-07-15 04:21:48,41.841448,-87.816588,17031.0
Florida,Miami-Dade,515765,6472,0.0,509293.0,2021-07-15 04:21:48,25.611236,-80.551706,12086.0
Texas,Harris,406425,6638,0.0,399787.0,2021-07-15 04:21:48,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,134653,132597,146942,14155,40055,1266,4662937,226459,31323,651804,...,5173972,376876,116421,4,285910,32665,315224,6952,176742,70426
2021-07-13,134653,132608,147883,14167,40138,1266,4682960,226597,31429,652022,...,5210472,377297,116918,4,286990,35409,315224,6961,178748,73271
2021-07-14,136643,132616,148797,14167,40327,1267,4702657,226756,31513,652354,...,5252655,377704,117409,4,288099,38239,315409,6964,180549,76381


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,5791,2456,3851,127,939,42,99255,4547,912,10723,...,128697,5833,774,1,3302,125,3582,1366,2867,2236
2021-07-13,5791,2456,3867,127,941,42,99640,4549,912,10724,...,128747,5849,778,1,3315,132,3582,1366,2904,2274
2021-07-14,5923,2456,3882,127,946,42,100250,4552,912,10726,...,128797,5854,781,1,3327,138,3585,1366,2949,2332


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,82586,130052,102010,13787,34357,1222,4300823,217731,23898,638965,...,15991,364431,111514,3,268872,9331,310027,4142,162239,46932
2021-07-13,82586,130053,102594,13796,34500,1222,4314593,217810,23907,639035,...,16007,365219,112018,3,269971,9553,310027,4147,163681,48102
2021-07-14,82586,130059,103192,13796,34584,1222,4326403,217896,23911,639154,...,16127,365852,112546,3,270996,9624,310239,4151,164924,49053


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7262,7262,7277,7277,7294,7299,7299,7306,7306,7320
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1476,1476,1478,1478,1478,1478,1478,1479,1479,1481
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9653,9653,9674,9674,9698,9703,9703,9720,9720,9754
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3144,3144,3146,3146,3147,3150,3150,3150,3150,3155
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16343,16343,16390,16390,16422,16448,16448,16454,16454,16483


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-12,555215,71905,0,901561,357473,3840251,563086,350039,110093,49,...,870971,3020886,420214,24489,4033,682856,456709,164625,678944,63138
2021-07-13,555215,71905,0,901906,358949,3845180,563642,350245,110112,49,...,871546,3024610,420685,24497,4043,683202,457814,164685,679180,63235
2021-07-14,556613,72153,0,903851,360258,3847746,564164,350386,110182,49,...,872362,3029676,421257,24516,4043,683614,458517,164763,679389,63331


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-07-12        7306   22337    2367  2704   7025    1250   2283   14841   
2021-07-13        7306   22337    2367  2704   7025    1250   2283   14841   
2021-07-14        7320   22464    2373  2708   7055    1250   2288   14861   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-12         3763     1896  ...    2910    704     3281      806   
2021-07-13         3763     1896  ...    2912    704     3281      809   
2021-07-14         3773     1898  ...    2917    707     3281      812   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-12           4853  3824  2329          0      929    664  
2021-07-13           4859  3826  2330          0      929    664  
2021-07-14           4869  3831  2332          0      929    664  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-12,11402,381,0,18035,5955,63964,6853,8279,1695,0,...,12608,52661,2399,258,32,11450,6000,2910,8185,751
2021-07-13,11402,381,0,18055,5970,63984,6861,8279,1695,0,...,12616,52690,2402,258,32,11459,6022,2909,8189,760
2021-07-14,11428,381,0,18076,5977,64012,6869,8279,1696,0,...,12625,52727,2406,258,32,11467,6027,2916,8196,760


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-12         113     318      60   65    139      42     71     331   
2021-07-13         113     318      60   65    139      42     71     331   
2021-07-14         113     321      60   65    139      42     71     331   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-12          123       47  ...      33     13       31        7   
2021-07-13          123       47  ...      33     14       31        7   
2021-07-14          124       47  ...      33     14       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-12             40    11    13          0       26      6  
2021-07-13             41    11    13          0       26      6  
2021-07-14             41    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,0.023308,0.000038,0.006011,0.005684,0.002428,0.00000,0.003225,0.000314,0.003267,0.000295,...,0.006773,0.000789,0.003950,0.0,0.007641,0.071757,0.000203,0.000576,0.005765,0.039268
2021-07-13,0.000000,0.000083,0.006404,0.000848,0.002072,0.00000,0.004294,0.000609,0.003384,0.000334,...,0.007055,0.001117,0.004269,0.0,0.003777,0.084004,0.000000,0.001295,0.011350,0.040397
2021-07-14,0.014779,0.000060,0.006181,0.000000,0.004709,0.00079,0.004206,0.000702,0.002673,0.000509,...,0.008096,0.001079,0.004200,0.0,0.003864,0.079923,0.000587,0.000431,0.010076,0.042445


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,0.041359,0.0,0.003910,0.0,0.004278,0.0,0.004798,0.000660,0.001098,0.000000,...,0.000047,0.003095,0.003891,0.0,0.007322,0.050420,0.000000,0.0,0.015946,0.023341
2021-07-13,0.000000,0.0,0.004155,0.0,0.002130,0.0,0.003879,0.000440,0.000000,0.000093,...,0.000389,0.002743,0.005168,0.0,0.003937,0.056000,0.000000,0.0,0.012905,0.016995
2021-07-14,0.022794,0.0,0.003879,0.0,0.005313,0.0,0.006122,0.000659,0.000000,0.000186,...,0.000388,0.000855,0.003856,0.0,0.003620,0.045455,0.000838,0.0,0.015496,0.025506


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,0.018612,0.000015,0.005718,0.001235,0.000874,0.0,0.004184,0.000335,0.000293,0.000119,...,0.005534,0.002440,0.002923,0.0,0.009920,0.006038,0.000378,0.000241,0.012633,0.028827
2021-07-13,0.000000,0.000008,0.005725,0.000653,0.004162,0.0,0.003202,0.000363,0.000377,0.000110,...,0.001001,0.002162,0.004520,0.0,0.004087,0.023792,0.000000,0.001207,0.008888,0.024930
2021-07-14,0.000000,0.000046,0.005829,0.000000,0.002435,0.0,0.002737,0.000395,0.000167,0.000186,...,0.007497,0.001733,0.004714,0.0,0.003797,0.007432,0.000684,0.000965,0.007594,0.019770


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-12,0.001100,0.002426,NaN,0.000135,0.005663,0.001247,0.000574,0.000200,0.000173,0.0,...,0.001402,0.001182,0.002955,0.001308,0.006991,0.000268,0.003529,0.000973,0.000501,0.002732
2021-07-13,0.000000,0.000000,NaN,0.000383,0.004129,0.001284,0.000987,0.000589,0.000173,0.0,...,0.000660,0.001233,0.001121,0.000327,0.002480,0.000507,0.002419,0.000364,0.000348,0.001536
2021-07-14,0.002518,0.003449,NaN,0.002157,0.003647,0.000667,0.000926,0.000403,0.000636,0.0,...,0.000936,0.001675,0.001360,0.000776,0.000000,0.000603,0.001536,0.000474,0.000308,0.001518


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-07-12      0.000959  0.003144  0.000846  0.000740  0.000285     0.0   
2021-07-13      0.000000  0.000000  0.000000  0.000000  0.000000     0.0   
2021-07-14      0.001916  0.005686  0.002535  0.001479  0.004270     0.0   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-12      0.001755  0.000539  0.000798  0.000000  ...  0.001376   
2021-07-13      0.000000  0.000000  0.000000  0.000000  ...  0.000687   
2021-07-14      0.002190  0.001348  0.002657  0.001055  ...  0.001717   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-12      0.002849  0.000915  0.002488   0.004138  0.002096  0.000000   
2021-07-13      0.000000  0.000000  0.003722   0.001236  0.000523  0.000429   
2021-07-14      0.004261  0.000000  0.003708   0.002058  0.001307  0.000858   

Province_State                             
Admin2         Unassigned Washakie Weston  
2021-07-12            NaN      0.0    0.0  
2021-07-13            NaN      0.0    0.0  
2021-07-14            NaN      0.0    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-12,0.00000,0.005277,NaN,0.000111,0.001177,0.000501,0.000000,0.0,0.00000,NaN,...,0.000953,0.000608,0.002507,0.0,0.0,0.000000,0.000500,0.000688,0.000367,0.000000
2021-07-13,0.00000,0.000000,NaN,0.001109,0.002519,0.000313,0.001167,0.0,0.00000,NaN,...,0.000635,0.000551,0.001251,0.0,0.0,0.000786,0.003667,-0.000344,0.000489,0.011984
2021-07-14,0.00228,0.000000,NaN,0.001163,0.001173,0.000438,0.001166,0.0,0.00059,NaN,...,0.000713,0.000702,0.001665,0.0,0.0,0.000698,0.000830,0.002406,0.000855,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-12         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2021-07-13         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2021-07-14         0.0  0.009434     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                              \
Admin2         Chambers Cherokee  ...    Park    Platte Sheridan Sublette   
2021-07-12      0.00000      0.0  ...     0.0  0.000000      0.0      0.0   
2021-07-13      0.00000      0.0  ...     0.0  0.076923      0.0      0.0   
2021-07-14      0.00813      0.0  ...     0.0  0.000000      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-12          0.000   0.0   0.0        NaN      0.0    0.0  
2021-07-13          0.025   0.0   0.0        NaN      0.0    0.0  
2021-07-14          0.000   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,0.013104,0.000046,0.005644,0.003093,0.002448,0.000031,0.002859,0.000435,0.003164,0.000265,...,0.006539,0.000928,0.004062,3.446586e-26,0.004636,0.070190,0.000214,0.000613,0.007269,0.031351
2021-07-13,0.006552,0.000064,0.006024,0.001970,0.002260,0.000016,0.003577,0.000522,0.003274,0.000300,...,0.006797,0.001022,0.004165,1.723293e-26,0.004207,0.077097,0.000107,0.000954,0.009310,0.035874
2021-07-14,0.010665,0.000062,0.006102,0.000985,0.003485,0.000403,0.003891,0.000612,0.002973,0.000405,...,0.007446,0.001051,0.004182,8.616465e-27,0.004036,0.078510,0.000347,0.000693,0.009693,0.039160


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,0.022532,3.303626e-09,0.003566,3.501645e-24,0.004242,1.946636e-18,0.004015,0.000598,0.000686,0.000030,...,0.000137,0.002658,0.004299,0.0,0.004703,0.048990,0.000223,0.000089,0.016870,0.024210
2021-07-13,0.011266,1.651813e-09,0.003860,1.750823e-24,0.003186,9.733179e-19,0.003947,0.000519,0.000343,0.000062,...,0.000263,0.002700,0.004734,0.0,0.004320,0.052495,0.000112,0.000044,0.014888,0.020602
2021-07-14,0.017030,8.259065e-10,0.003870,8.754113e-25,0.004250,4.866590e-19,0.005034,0.000589,0.000172,0.000124,...,0.000326,0.001778,0.004295,0.0,0.003970,0.048975,0.000475,0.000022,0.015192,0.023054


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-12,0.014825,0.000021,0.005337,0.000693,0.001397,2.000373e-07,0.004256,0.000356,0.000535,0.000125,...,0.003057,0.002586,0.003252,8.271806e-25,0.006123,0.010872,0.000504,0.000891,0.014921,0.024358
2021-07-13,0.007412,0.000015,0.005531,0.000673,0.002780,1.000186e-07,0.003729,0.000359,0.000456,0.000117,...,0.002029,0.002374,0.003886,4.135903e-25,0.005105,0.017332,0.000252,0.001049,0.011904,0.024644
2021-07-14,0.003706,0.000030,0.005680,0.000336,0.002607,5.000932e-08,0.003233,0.000377,0.000312,0.000152,...,0.004763,0.002054,0.004300,2.067952e-25,0.004451,0.012382,0.000468,0.001007,0.009749,0.022207


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-12,0.000848,0.001377,NaN,0.000505,0.003208,0.000859,0.000719,0.000144,0.000291,1.363120e-146,...,0.000773,0.000783,0.001663,0.000700,0.004350,0.000321,0.001902,0.000534,0.000280,0.001580
2021-07-13,0.000424,0.000688,NaN,0.000444,0.003669,0.001071,0.000853,0.000366,0.000232,6.815602e-147,...,0.000717,0.001008,0.001392,0.000514,0.003415,0.000414,0.002161,0.000449,0.000314,0.001558
2021-07-14,0.001471,0.002069,NaN,0.001300,0.003658,0.000869,0.000890,0.000384,0.000434,3.407801e-147,...,0.000827,0.001341,0.001376,0.000645,0.001707,0.000508,0.001848,0.000462,0.000311,0.001538


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2021-07-12      0.000744  0.002102  0.000867  0.000544  0.000332   
2021-07-13      0.000372  0.001051  0.000434  0.000272  0.000166   
2021-07-14      0.001144  0.003368  0.001484  0.000876  0.002218   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2021-07-12      4.441186e-07  0.001332  0.000512  0.000653  0.000456  ...   
2021-07-13      2.220593e-07  0.000666  0.000256  0.000327  0.000228  ...   
2021-07-14      1.110297e-07  0.001428  0.000802  0.001492  0.000642  ...   

Province_State   Wyoming                                                     \
Admin2              Park    Platte  Sheridan  Sublette Sweetwater     Teton   
2021-07-12      0.000731  0.002169  0.000492  0.001595   0.002371  0.001221   
2021-07-13      0.000709  0.001085  0.000246  0.002659   0.001804  0.000872   
2021-07-14      0.001213  0.002673  0.000123  0.003184   0.001931  0.001089   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2021-07-12      0.000068       -1.0  0.000068  0.000144  
2021-07-13      0.000248       -1.0  0.000034  0.000072  
2021-07-14      0.000553       -1.0  0.000017  0.000036  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-12,0.000145,0.002971,NaN,0.000283,0.000720,0.000469,0.000508,1.951700e-06,2.963201e-07,NaN,...,0.000530,0.000451,0.001422,2.853601e-06,0.001042,0.000081,0.000495,0.000469,0.000406,0.000042
2021-07-13,0.000072,0.001486,NaN,0.000696,0.001619,0.000391,0.000838,9.758498e-07,1.481600e-07,NaN,...,0.000582,0.000501,0.001336,1.426801e-06,0.000521,0.000433,0.002081,0.000063,0.000448,0.006013
2021-07-14,0.001176,0.000743,NaN,0.000930,0.001396,0.000414,0.001002,4.879249e-07,2.950593e-04,NaN,...,0.000648,0.000602,0.001501,7.134003e-07,0.000260,0.000566,0.001456,0.001235,0.000651,0.003007


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                                  \
Admin2               Autauga   Baldwin       Barbour      Bibb        Blount   
2021-07-12      1.483847e-13  0.000642  1.616397e-08  0.000244  1.272479e-20   
2021-07-13      7.419235e-14  0.000321  8.081986e-09  0.000122  6.362395e-21   
2021-07-14      3.709617e-14  0.004878  4.040993e-09  0.000061  3.181197e-21   

Province_State                                                                \
Admin2               Bullock        Butler   Calhoun      Chambers  Cherokee   
2021-07-12      4.231298e-20  1.047241e-16  0.000047 -9.843793e-07  0.002778   
2021-07-13      2.115649e-20  5.236207e-17  0.000024 -4.921897e-07  0.001389   
2021-07-14      1.057824e-20  2.618104e-17  0.000012  4.064795e-03  0.000694   

Province_State  ...   Wyoming                                        \
Admin2          ...      Park    Platte      Sheridan      Sublette   
2021-07-12      ...  0.000248  0.000005  5.016463e-38  2.203777e-58   
2021-07-13      ...  0.000124  0.038464  2.508232e-38  1.101889e-58   
2021-07-14      ...  0.000062  0.019232  1.254116e-38  5.509443e-59   

Province_State                                                       \
Admin2            Sweetwater         Teton         Uinta Unassigned   
2021-07-12      9.552026e-11  1.293504e-11  7.058608e-23   0.752941   
2021-07-13      1.250000e-02  6.467518e-12  3.529304e-23   0.752941   
2021-07-14      6.250000e-03  3.233759e-12  1.764652e-23   0.752941   

Province_State                              
Admin2              Washakie        Weston  
2021-07-12      1.369244e-50  1.164153e-11  
2021-07-13      6.846220e-51  5.820766e-12  
2021-07-14      3.423110e-51  2.910383e-12  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
